# The BEST Cardiac segmentation Challenge 2019

Welcome to the medical image segmentation challenge for BEST Leuven SC 2019. Today, you and your team will have to solve a typical medical image segmentation task where you will have to segment the cardiac muscle, called myocardium, on cardiac T1 weighted images as shown below.

<img src="data/demo/example_contours.png">

In this Jupyter notebook, you will find a simple python implementation of a CNN to solve this task. By just running this script, you will be able to train a network that is capable of segmenting the myocardium. However, you will notice that the performance of this network is not very good. It is your task to improve this code and to try out new things in order get this performance as high as possible. At the end of today, the team with the highest mean DICE score for the test set, wins.

## Introducing the dataset
In this challenge, we will use cardiac MR images showing the heart in short-axis orientation (see figure). MRI measures the magnetic properties of the tissue and is excellent for discrimination of soft tissues. For the acquisition of an image, the magnetic equilibrium in the tissue is disturbed by a radiofrequency pulse whereafter two different relaxation processes start: T1 relaxation and T2 relaxation. The MR image is created from the signal measured during relaxation. By changing acquisition parameters such as the timing between equilibrium disturbance and signal measurement, different image contrasts can be obtained that emphasize a particular property of the tissue. Examples are T1-weighted or T2-weighted images. 

<img src="data/demo/short-axis.png">

The dataset that's being used for this challenge is publicly available [1] and contains cardiac T1-weighted images for 210 patients (11 T1-weighted image x 5 slices per patient). In clinical practice, these images are used to construct T1 maps which characterize the myocardium in a quantitative way. This allows to detect diffuse deposition of substances in the myocardium, e.g. proteins, collagen, lipids,… which are characteristic for specific cardiac diseases. In the figure below, you can see an example T1 map of a healthy heart on the left and of a diseased heart with increased T1 on the right.

<img src="data/demo/T1maps.png">

The benefit of automatic myocardial segmentation in the T1-weighted images is twofold: 1) automatic calculation of an average myocardial T1 value and 2) correction of motion between different T1-weighted images to improve T1 map accuracy. Furthermore, the problem of automatic myocardial segmentation in other MR modalities is very similar and allows calculation of additional structural and functional parameters of the heart. An example is the evolution myocardial thickness during the cardiac cycle which can be calculated from cine images.

From the dataset in [1], we selected for 196 patients one image of the middle slices and subsampled and cropped the image to obtain 128x128 images with a voxelsize of 2.26x2.26mm$^2$. This reduction of the dataset was done to allow faster training. The dataset is divided in 150 images for training, 23 for validation and 23 for testing. Manual delineations of the myocardium for each T1-weighted image are provided as a binary map where each background pixel has value 0 and the myocardium pixels have value 1.

<img src="data/demo/example_groundtruth.png">

## Import the required packages
Do NOT remove any of these lines unless you know what you're doing! ;)

In [ ]:
import os
import numpy as np
import glob
import random
import numpy as np

import nibabel as nib
from skimage import measure
import cv2

from matplotlib import pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from keras.layers import Dropout, BatchNormalization
from keras.optimizers import Adam

from utils.losses import dice
from utils.metrics import IoU, dice_coef
from utils.evaluation import get_iou, get_dice

import imgaug as ia
from imgaug import augmenters as iaa

### ADD your own import under this line ###
from keras import regularizers

## Define the model
In this block, you are given the basic definition of a very popular CNN for medical image segmentation, called Unet [2]. You are free to adapt this to your own will. Here are some suggestions that you can consider:
- Add or remove layers (e.g. [dropout layers](https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5) and [batch normalization](https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c) layers)
- Change the feature depth, [kernel size](https://blog.sicara.com/about-convolutional-layer-convolution-kernel-9a7325d34f7d), [activation function](https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0), padding type, ...
- Use an entirely different architecture (Check out [this review](https://medium.com/@arthur_ouaknine/review-of-deep-learning-algorithms-for-image-semantic-segmentation-509a600f7b57) article for some inspiration)

In [ ]:
def unet(input_size=(128, 128)):
    inputs = Input(input_size + (1,))
    
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv4))
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    up6 = Conv2D(32, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv5))
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(32, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(32, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(16, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(16, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(16, 3, activation='relu', padding='same')(conv7)

    conv8 = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(input=inputs, output=conv8)

    return model

## Create data generator
### Preprocessing
When using images for traditional image analysis techniques, extensive preprocessing steps are typically required to reduce noise, fix heterogeneous lighting, ignore artefacts, etc. It turns out that this is not necessary when training a neural network with sufficient data. There is however one preprocessing step that is very useful: normalization/standardization of the input data. Even though not strictly necessary, this will make training faster and more robust. There are many different approaches to standardize the input data, but here we simply subtract the mean pixel value and divide by its standard deviation for each image.

### Augmentation
In general computer vision tasks, there are usually several thousands or even millions of annotated images available to train a neural network. In medical imaging, this is rarely the case and we have to deal with very small datasets (typically ~10-100 cases). A common approach to alleviate this issue is [data augmentation](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced), a technique to artificially increase the number of training samples. Plausible new training images are generated by transforming the existing images. In this example, we perform very basic data augmentation by randomly flipping the images from the training set. Think of some other plausible transformations that you could apply and implement them here. Data augmentation works!

We use a toolbox called [imgaug](https://imgaug.readthedocs.io/en/latest/) which makes it very easy to add different augmentation techniques.

### Batch pair generation
Once we are able to preprocess and augment images, it is time to feed the images to the neural network for training. For this, we need a generator that will continuously create batches of pairs of images. In each iteration of the training process, we will feed a bunch of images (calles a batch) and their ground truths to the network. The number of images in a batch is typically a power of 2 and should be large enough such that the optimizer can make a good update for the weights in the network, but it can't be infinitely high because we have a limited amount of memory in a GPU. Additionally, it's been shown that the noise introduced by a smaller batch size can actually be beneficial and provide a sort of regularization effect. A batch size of 16 is an acceptable choice, but maybe you can do better? You can specify this number later when starting the training process.

In [ ]:
def preprocess(img):
    return (img - np.mean(img)) / np.std(img)


augmenter = iaa.Sequential([
                iaa.Fliplr(0.5)])

### Do NOT change lines below unless you know what you're doing! ###
def augment(img, gt):
    augmenter_det = augmenter.to_deterministic()
    img = augmenter_det.augment_image(img).astype('float32')
    gt = augmenter_det.augment_image(gt)
    return img, gt

def image_generator(path, batch_size=16, input_size=(128, 128), do_augment=False):
    img_fns = glob.glob(path + 'Patient_*[0-9].nii')
    while True:
        batch_fns = np.random.choice(a=img_fns, size=batch_size)

        batch_imgs = []
        batch_gts = []

        for fn in batch_fns:
            img = nib.load(fn).get_fdata()
            gt = nib.load(fn.replace('.nii', '_gt.nii')).get_fdata()

            img = cv2.resize(img, input_size)
            gt = cv2.resize(gt, input_size)

            img = np.expand_dims(img, axis=-1)
            gt = np.expand_dims(gt, axis=-1)
            
            img = preprocess(img)
            if do_augment:
                img, gt = augment(img, gt)

            batch_imgs += [img]
            batch_gts += [gt]

        yield(np.array(batch_imgs), np.array(batch_gts))

## Train the model
Now that you've defined your model and data generator, it's time to start training your network. First you'll need to create an instance of your network model, compile it and then start feeding data to it in order to update its weights and to hopefully end up with a well trained model.

### Compilation
Compilation of the model is important because here you will decide which optimizer and loss function you will use. You are free to change these two.

### Training
During training, a **batch of images** will be forwarded through the network, the result is then compared with the ground truth and based on the loss function and optimizer, the weights inside the network will be updated. This is one optimization **step**. Every next step, the loss value should be slightly lower than before (although some noise can be present, so don't be discouraged if the loss increases shortly). After a few of these steps, we will evaluate the trained model on the validation set to test if the loss is also lower on unseen data. Every time this is done, one **epoch** has passed. In theory, you can do this at every iteration step but then you're just unnecessarily slowing down the training because there's no need to evaluate this frequently.

You must now define the batch size, number of steps per epoch and number of epochs. Make sure that you train long enough such that your network has the time to converge to a good result. Training too long on the other hand takes time and might cause overfitting. So make sure to check out the training graphs after training to see if you should increase or decrease these numbers (leave the default values for the first time).

In [ ]:
input_size = (128, 128)
model = unet(input_size=input_size)
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy', dice_coef])

history = model.fit_generator(image_generator('data/train/', batch_size=32, input_size=input_size, do_augment=True),
                    steps_per_epoch=20,
                    epochs=100,
                    validation_data=image_generator('data/val/', batch_size=32, input_size=input_size),
                    validation_steps=5,
                    verbose=1)

## Visualize training progress
Here you can plot the loss of the model in a more visual manner. This is a very important step and will help you to determine whether your model is [under- or overfitting](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76).

If your network is underfitting, you should probably look into expanding your network with more filters or layers. Or alternatively, you can look for a different, more complex network architecture. 

If your network is overfitting, there are [many strategies](https://towardsdatascience.com/preventing-deep-neural-network-from-overfitting-953458db800a) that you can try to correct for this: 
- You can decrease the capacity of your network by using less filters/layers
- You can use L1 or L2 regularization
- You can try Dropout and/or batch normalization
- You can adopt Early stopping

In [ ]:
### DO NOT CHANGE THIS CELL ###
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Save and load model
You can save a trained model and load it again later for testing its performance.

HINT: You can also load an already trained model and train it further (possibly with a different learning rate or optimizer). This way, you don't have to start from scratch when all you want to do is train your network longer. 

In [ ]:
model.save('data/models/test.h5')

In [ ]:
del model  # just to show that loading on the next line actually works
model = load_model('data/models/test.h5', custom_objects={'dice_coef': dice_coef})

## Test your model
### Qualitatively
With this piece of code, you can quickly visualize a few predictions of the network on the test set. If all went well, the prediction and ground truth should look similar.

In [ ]:
### DO NOT CHANGE THIS CELL ###
for i in range(4):
    patient = random.randint(174, 196)
    img = nib.load('/uz/data/avalok/mic/tmp/teelbo0/BEST_SC2019/data/test/Patient_' + str(patient) + '.nii').get_fdata()
    gt = nib.load('/uz/data/avalok/mic/tmp/teelbo0/BEST_SC2019/data/test/Patient_' + str(patient) + '_gt.nii').get_fdata()
    img = preprocess(img)
    img = cv2.resize(img, input_size)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)

    gt = np.expand_dims(gt, axis=-1)
    result = model.predict(img)

    plt.figure(num=str(i), figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.subplot(141)
    plt.imshow(img[0, :, :, 0])
    plt.title('Patient_' + str(patient))
    plt.subplot(142)
    plt.imshow(result[0, :, :, 0])
    plt.title('Prediction')
    plt.subplot(143)
    plt.imshow(result[0, :, :, 0] > 0.5)
    plt.title('Thresholded')
    plt.subplot(144)
    plt.imshow(gt[:, :, 0])
    plt.title('Ground truth')
    plt.show()

### Quantitatively
With this piece of code, you can calculate the performance of your trained network on the entire test set. This is the number that will define your ranking in the competition.

In [ ]:
### DO NOT CHANGE THIS CELL ###
dices = []
for patient in range(174, 197):
    img = nib.load('/uz/data/avalok/mic/tmp/teelbo0/BEST_SC2019/data/test/Patient_' + str(patient) + '.nii').get_fdata()
    gt = nib.load('/uz/data/avalok/mic/tmp/teelbo0/BEST_SC2019/data/test/Patient_' + str(patient) + '_gt.nii').get_fdata()
    
    img = preprocess(img)
    img = cv2.resize(img, input_size)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)

    gt = cv2.resize(gt, input_size)
    gt = np.expand_dims(gt, axis=-1)
    gt = np.expand_dims(gt, axis=0)
    result = model.predict(img)
    
    dices.append(get_dice(gt, result))
    
print("The mean DICE score for the test set is: " + str(np.mean(dices)))

## References
[1] [Hossam El‐Rewaidy, Maryam Nezafat, Jihye Jang, Shiro Nakamori, Ahmed S. Fahmy, and Reza Nezafat. "Nonrigid active shape model–based registration framework for motion correction of cardiac T1 mapping." Magnetic resonance in medicine (2018), doi: 10.1002/mrm.27068](https://cardiacmr.hms.harvard.edu/downloads-0)

[2] [Ronneberger, O., Fischer, P. and Brox, T., 2015, October. U-net: Convolutional networks for biomedical image segmentation. In International Conference on Medical image computing and computer-assisted intervention (pp. 234-241). Springer, Cham.](https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28)